# Development practises


## Topics
- naming conventions
- pretty functions
- multiple dispatch
- variable scope
    - arrays as an exception
- metaprogramming (see Bonus notebook)

## Naming conventions in Julia

- Word separation can be indicated by underscores (`_`), but use of underscores is discouraged unless the name would be hard to read otherwise.
- Names of Types begin with a capital letter and word separation is shown with CamelCase instead of underscores.
- Names of functions and macros are in lower case, without underscores.  
- Functions that modify their inputs have names that end in `!`. These functions are sometimes called mutating functions or in-place functions.


## Optimizing nested loops: index "convention"

Julia is a column-major programming language. The inner loop should concern rows rather than columns. This is due to how arrays are stored in memory.

What this means is: **first index changes fastest**.
```julia
for k in 1:Nz
    for j in 1:Ny
        for i in 1:Nx
            arr[i,j,k]
        end
    end
end
```

In [1]:
# This function is badly written, because it looks at every column of a row, then at 
# every column of the next row, and so on

function laplacian_bad(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ir = 2:nr-1 
        for ic = 2:nc-1 # bad loop nesting order
            lap_x[ir,ic] =
                (x[ir+1,ic] + x[ir-1,ic] +
                x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
        end
    end
end

laplacian_bad (generic function with 1 method)

In [2]:
# In this version, the two loops are nested properly:

function laplacian_good(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ic = 2:nc-1
        for ir = 2:nr-1 # good loop nesting order
            lap_x[ir,ic] =
                (x[ir+1,ic] + x[ir-1,ic] +
                x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
        end
    end
end

laplacian_good (generic function with 1 method)

## Let's see the effect in practise!

In [4]:
using Printf
function main_test(nr, nc)
    field = zeros(nr, nc)
    for ic = 1:nc, ir = 1:nr
        if ir == 1 || ic == 1 || ir == nr || ic == nc
            field[ir,ic] = 1.0
        end
    end
    lap_field = zeros(size(field))

    time = @elapsed laplacian_bad(lap_field, field)
    @printf "laplacian_bad:          %.3f s\n" time
    
    time = @elapsed laplacian_good(lap_field, field)
    @printf "laplacian_good:         %.3f s\n" time

end
main_test(10^4, 10^4)

laplacian_bad:          3.138 s
laplacian_good:         0.265 s


## Making functions pretty: optional arguments
You can define functions with optional arguments, so that the function can use sensible defaults if specific values aren't supplied. You provide a default symbol and value in the argument list

In [ ]:
function xyzpos(x, y, z=0)
    println("$x, $y, $z")
end

In [6]:
xyzpos(0,0)
xyzpos(0,0,1)

0, 0, 0
0, 0, 1


## Making functions pretty: keywords
When you write a function with a long list of arguments like this:
```julia
function f(p, q, r, s, t, u)
...
end
```
sooner or later, you will forget the order in which you have to supply the arguments. 

You can avoid this problem by using keywords to label arguments. Use a semicolon (`;`) after the function's unlabelled arguments, and follow it with one or more keyword=value pairs:

In [ ]:
function f(p, q ; r = 4, s = "hello")
  println("p is $p")
  println("q is $q")
  return "r => $r, s => $s"
end
f(1,2)
f("a", "b", r=pi, s=22//7)

## Advanced: Functions with variable number of arguments
Functions can be defined so that they can accept any number of arguments:

In [ ]:
function fvar(args...)
    println("you supplied $(length(args)) arguments")
    for arg in args
       println(" argument ", arg)
    end
end
fvar()
fvar(64)
fvar(64, 64, 55)

The three dots indicate the **splat**. Here it means "any", including "none". 

## Multiple dispatch
Until now, we have, in our examples, defined only functions with a single method having unconstrained argument types. 

Such functions behave just like they would in traditional dynamically typed languages. Nevertheless, we have used multiple dispatch and methods almost continually without being aware of it: all of Julia's standard functions and operators have many methods defining their behavior over various possible combinations of argument type and count.

This is known as multiple dispatch!

When defining a function, one can optionally constrain the types of parameters it is applicable to, using the `::` type-assertion operator

In [ ]:
function myfunc(x::Float64, y::Float64) 
    2x + y
end

This function definition applies only to calls where `x` and `y` are both values of type `Float64`

Applying it to any other types of arguments will result in a `MethodError`.

The arguments must be precisely of type `Float64`. Other numeric types, such as integers or 32-bit floating-point values, are not automatically converted to 64-bit floating-point, nor are strings parsed as numbers. 

Because `Float64` is a concrete type and concrete types cannot be subclassed in Julia, such a definition can only be applied to arguments that are exactly of type `Float64`. 

It may often be useful, however, to write more general methods where the declared parameter types are abstract:

In [10]:
function myfunc(x::Number, y::Number)
    2x + y
end

myfunc (generic function with 2 methods)

In [11]:
myfunc(2.0, 3)

7.0

You can easily see which methods exist for a function by entering the function object itself in an interactive session:

In [12]:
myfunc

myfunc (generic function with 2 methods)

This output tells us that `myfunc` is a function object with two methods. To find out what the signatures of those methods are, use the `methods()` function:

In [13]:
methods(myfunc)

# 2 methods for generic function "myfunc":
[1] myfunc(x::Float64, y::Float64) in Main at In[9]:2
[2] myfunc(x::Number, y::Number) in Main at In[10]:2

## Advanced: Parametric methods
Method definitions can optionally have type parameters qualifying the signature:

In [14]:
function same_type(x::T, y::T) where {T}
    true
end

function same_type(x,y)
    false
end

same_type (generic function with 2 methods)

The first method applies whenever both arguments are of the same concrete type, regardless of what type that is, while the second method acts as a catch-all, covering all other cases. Thus, overall, this defines a boolean function that checks whether its two arguments are of the same type

In [15]:
same_type(1,2)

true

In [16]:
same_type(1, 2.0)

false

In [17]:
same_type(1.0, 2.0)

true

In [18]:
same_type("foo", 2.0)

false

In [19]:
same_type(Int32(1), Int64(2))

false

## Scope of variables
- Global scope
    - Module spesific (namespaces)
- Local scopes
    - functions, for's, while's,...

## Local scope
A new local scope is introduced by most code-blocks.
    
A local scope usually inherits all the variables from its parent scope, both for reading and writing. 

A newly introduced variable in a local scope does not back-propagate to its parent scope. For example, here the z is not introduced into the top-level scope:

In [20]:
for i = 1:10
    z = 1
end
z

UndefVarError: UndefVarError: z not defined

Function definitions are also in their own local scope. 

They do, however, inherit from their parent scope.

In [21]:
x, y = 1, 2
function foo()
    x = 2 #assignment introduces a new local
    return x + y # y refers to the global scope!
end

foo (generic function with 1 method)

In [22]:
foo()

4

In [23]:
x

1

An explicit `global` is needed to assign to a global variable:

In [ ]:
x = 1
function foobar()
    global x = 2
end

In [25]:
foobar()

2

In [26]:
x

2

## Exception: Arrays are always global
There is an important exception to these rules: arrays.

Changing elements of arrays is always done on the global scope. 

In [ ]:
arr = [1,2,3]
function oops()
    arr[2] = 10
    
    return "woops"
end

In [28]:
oops()

"woops"

In [29]:
arr

3-element Array{Int64,1}:
  1
 10
  3

## Constants
A common use of variables is giving names to specific, unchanging values. 

Such variables are only assigned once. This intent can be conveyed to the compiler using the `const` keyword:

In [ ]:
const e  = 2.71828182845904523536

It is difficult for the compiler to optimize code involving global variables, since their values (or even their types) might change at almost any time. If a global variable will not change, adding a const declaration solves this performance problem.

## Summary
- Writing pretty code is a good thing
    - see also the official [style guide](https://docs.julialang.org/en/v1/manual/style-guide/)
- remember the index ordering in loops!
    - first index changes fastest
- take advantage of multiple dispatch
    - this is what makes Julia fast